In [19]:
## Copy pretrained model file at \weights\ from "https://drive.google.com/drive/folders/1Srf-WYUixK0wiUddc9y3pNKHHno5PN6R".

from PIL import Image
import os
import numpy as np

In [20]:
def to_transparent(src_image):
    """
    Make the input image file transparent.
    :param src_image:
    :return: transparent image file.
    """
    threshold=100
    dist=5
    img=src_image.convert('RGBA')
    arr=np.array(np.asarray(img))
    r,g,b,a=np.rollaxis(arr,axis=-1)
    mask=((r>threshold)
          & (g>threshold)
          & (b>threshold)
          & (np.abs(r-g)<dist)
          & (np.abs(r-b)<dist)
          & (np.abs(g-b)<dist)
          )
    arr[mask,3]=0
    img=Image.fromarray(arr,mode='RGBA')
    return img

In [21]:
# Color transition.
color_styles = [
    (255,0,255, 255), # magenta
    (255,25,225, 255 ),
    (255,50,200, 255 ),
    (255,75,175, 255 ),
    (255,100,150, 255 ),
    (255,125,125, 255 ),
    (255,150,100, 255 ),
    (255,175,75, 255 ),
    (255,200,50, 255 ),
    (255,225,25, 255 ),
    (255,255,0, 255 ), # yellow
    (225,255,25, 255 ),
    (200,255,50, 255 ),
    (175,255,75, 255 ),
    (150,255,100, 255 ),
    (125,255,125, 255 ),
    (100,255,150, 255 ),
    (75,255,175, 255 ),
    (50,255,200, 255 ),
    (25,255,225, 255 ),
    (0,255,255, 255 ), # cyan
    (25,225,255, 255 ),
    (50,200,255, 255 ),
    (75,175,255, 255 ),
    (100,150,255, 255 ),
    (125,125,255, 255 ),
    (150,100,255, 255 ),
    (175,75,255, 255 ),
    (200,50,255, 255 ),
    (225,25,255, 255 ),
]

In [22]:
from data import tensor_to_img, read_img_path
from model import create_model
import torch

class ModelSingleton:
    """
    Use singleton to reduce initialization model
    """
    _model = None
    @staticmethod
    def get_model():
        if ModelSingleton._model is None:
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            ModelSingleton._model = create_model().to(device)
            ModelSingleton._model.eval()
        return ModelSingleton._model

def create_sketch(image_file_path: str) -> Image:
    """
    Create sketch.
    :param image_file_path:
    :return: sketch
    """
    model = ModelSingleton.get_model()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    img, aus_resize = read_img_path(image_file_path, 1024)
    aus_tensor = model(img.to(device))
    aus_img = tensor_to_img(aus_tensor)
    return Image.fromarray(aus_img), (1024, 1024)

In [23]:
def convert_color(image, color_rgba):
    """
    Convert color to passed color_rgba but white or transparent pixel.
    :param image:
    :param color_rgba:
    :return:
    """
    if image.mode != "RGBA":
        image=image.convert("RGBA")
    w,h=image.size
    for x in range(w):
        for y in range(h):
            r,g,b,a=image.getpixel((x,y))
            if (r == 0 and g == 0 and b == 0)or a == 0:
                continue
            image.putpixel((x,y), color_rgba)
    return image


In [24]:
def create_gaming_gif(original_file_path: str, dest_dir_path: str):
    """
    Make GAMING animated gif.
    :param original_file_path: source image file path.
    :param dest_dir_path: destination directory path.
    """

    # 線画を作る
    sketch,aus_resize = create_sketch(original_file_path)

    resize_data = aus_resize
    # 二回もファイル開いてて非効率だけど直すの面倒なので一旦このまま
    base_image = Image.open(original_file_path)
    base_image = base_image.resize(resize_data, Image.Resampling.LANCZOS)

    gif_sources = []
    file_name = os.path.basename(original_file_path)

    for color in color_styles:
        # 線画をコピーして着色
        mask_image = sketch.copy()
        mask_image = to_transparent(mask_image)
        mask_image = convert_color(mask_image, color)

        # RGBAを設定（合成するために、色がついてる部分以外は透過する
        target_image=base_image.copy().convert('RGBA')
        target_image.paste(mask_image, (0, 0), mask_image)

        # gifの元データとして追加
        gif_sources.append(target_image)

    gif_sources[0].save(f"{dest_dir_path}/{file_name}.gif",
           save_all=True, append_images=gif_sources[1:], optimize=False, duration=40, loop=0)


In [25]:
# Set source image file path as the 1st parameter and destination dir path as the 2nd.
create_gaming_gif(original_file_path=r".\src_files\\localstingray.jpg", dest_dir_path=r".\dest_images\\")